In [72]:
import sys
sys.path.append("..")

import pyodbc
from Derivados.DerivadosTipos.DerivadosSCC import *
from Derivados.DerivadosTipos.DerivadosFWD import *
from Derivados.DerivadosTipos.DerivadosSUC import *

import datetime
import pandas as pd
from Bonos.Correlaciones import ewma, ewma_new_new_pivotes, covarianza_pivotes
from Bonos.LibreriasUtiles.UtilesValorizacion import diferencia_dias_convencion


In [23]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'

cnn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

In [94]:
def creacion_derivado(FechaEfectiva, FechaVenc, AjusteFeriados, NocionalActivo, MonedaActivo,\
                                                     MonedaBase, TipoTasaActivo, TipoTasaPasivo, TasaActivo,\
                                                                        TasaPasivo, FrecuenciaActivo, FrecuenciaPasivo, \
                                                                        ID, Tipo, ID_Key=None): 

    info_derivado = dict()
    info_derivado["Tipo"] = str(Tipo)
    info_derivado["ID_Key"] = ""
    info_derivado["Administradora"] = "Admin"
    info_derivado["Fondo"] = "Fondo"
    info_derivado["Contraparte"] = "Contraparte"
    info_derivado["ID"] = int(ID)
    info_derivado["Nemotecnico"] = ""
    info_derivado["Mercado"] = "Local" 

    fecha = datetime.date(2018, 4, 18)
    hora = '1700'

    info_derivado["FechaEfectiva"] = str(FechaEfectiva)
    info_derivado["FechaVenc"] = str(FechaVenc)

    info_derivado["AjusteFeriados"] = str(AjusteFeriados)

    info_derivado["NocionalActivo"] = float(NocionalActivo)

    info_derivado["MonedaActivo"] = str(MonedaActivo)
    info_derivado["MonedaBase"] = str(MonedaBase)

    info_derivado["TipoTasaActivo"] = str(TipoTasaActivo)
    info_derivado["TipoTasaPasivo"] = str(TipoTasaPasivo)

    info_derivado["TasaActivo"] = float(TasaActivo)
    info_derivado["TasaPasivo"] = float(TasaPasivo)

    info_derivado["FrecuenciaActivo"] = str(FrecuenciaActivo)
    info_derivado["FrecuenciaPasivo"] = str(FrecuenciaPasivo)

    info1 = pd.DataFrame([info_derivado])
    print(Tipo.values)
    if str(Tipo.values[0]) == "SCC":
        derivado = DerivadosSCC(fecha, hora, info1, cnn)
    elif str(Tipo.values[0]) == "FWD":
        derivado = DerivadosFWD(fecha, hora, info1, cnn)
    elif str(Tipo.values[0]) == "SUC":
        derivado = DerivadosSUC(fecha, hora, info1, cnn)

    return derivado

#derivado1 = creacion_derivado("12/11/2019", "12/11/2021", "CL", 10*(10**9), "CLP", "CLP", "Fija", "Flotante", 1.45, -1000, "Semi annual", "Semi annual", 3, "SCC")

#derivado1.genera_flujos()
#derivado1.valoriza_flujos()



In [95]:
def extrar_crear_derivado(ID_Key):

    derivado = ("SELECT * FROM [dbDerivados].[dbo].[TdCarteraDerivados_V2] WHERE ID_Key = '"+str(ID_Key) + "'")
    derivado = pd.read_sql(derivado, cnn)
    
    FechaEfectiva = derivado["FechaEfectiva"][0]
    FechaEfectiva = str(FechaEfectiva).split("-")
    dia = FechaEfectiva[2].split(" ")[0]
    FechaEfectiva = str(dia + "/" + str(FechaEfectiva[1]) +"/"+ str(FechaEfectiva[0]))

    FechaVenc = derivado["FechaVenc"][0]
    FechaVenc = str(FechaVenc).split("-")
    dia = FechaVenc[2].split(" ")[0]

    FechaVenc = str(dia + "/" + str(FechaVenc[1]) +"/"+ str(FechaVenc[0]))

    AjusteFeriados = derivado["AjusteFeriados"][0]
    NocionalActivo = derivado["NocionalActivo"][0]
    MonedaActivo = derivado["MonedaActivo"][0]
    MonedaBase = derivado["MonedaBase"][0]
    TipoTasaActivo = derivado["TipoTasaActivo"][0]
    TipoTasaPasivo = derivado["TipoTasaPasivo"][0]
    TasaActivo = derivado["TasaActivo"][0]
    TasaPasivo = derivado["TasaPasivo"][0]
    FrecuenciaActivo = derivado["FrecuenciaActivo"][0]
    FrecuenciaPasivo = derivado["FrecuenciaPasivo"][0]
    ID = derivado["ID"]
    Tipo = derivado["Tipo"]
    ID_Key = derivado["ID_Key"]

    dev = creacion_derivado(FechaEfectiva, FechaVenc, AjusteFeriados, NocionalActivo, MonedaActivo,\
                                                     MonedaBase, TipoTasaActivo, TipoTasaPasivo, TasaActivo,\
                                                                        TasaPasivo, FrecuenciaActivo, FrecuenciaPasivo, \
                                                                        ID, Tipo, ID_Key=ID_Key)

    return dev

C = extrar_crear_derivado("146883")
C.genera_flujos()
C.valoriza_flujos()
C.flujos_valorizados[["ID","ActivoPasivo", "Fecha", "FechaFixing", "FechaFlujo", "FechaPago", "Flujo", "ValorPresenteMonFlujo", "Moneda", "MonedaBase"]]

['FWD']


Exception: ERROR: No viene la columna 'NocionalPasivo'

In [38]:
def seleccionar_curva_derivados(moneda):

    curva = ("SELECT * FROM [dbDerivados].[dbo].[TdCurvasDerivados] WHERE Tipo = 'CurvaEfectiva_"+ moneda +"' AND Hora = '1700' ORDER BY Fecha ASC")
    curva = pd.read_sql(curva, cnn)
    return curva

def seleccionar_curva_fecha(moneda, fecha):

    curva = ("SELECT * FROM [dbDerivados].[dbo].[TdCurvasDerivados] WHERE Tipo = 'CurvaEfectiva_"+ moneda +"' AND Hora = '1700' AND Fecha = '"+fecha+"' ORDER BY Fecha ASC")
    curva = pd.read_sql(curva, cnn)
    return curva

a = seleccionar_curva_derivados("CLP")
a

,Fecha,Hora,Tipo,Curva
0,2014-10-07,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,999913600351922|7#0,999389..."
1,2014-10-15,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,999908750709722|7#0,999355..."
2,2014-10-16,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,999897123411671|7#0,999274..."
3,2014-10-17,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|3#0,999806150349667|7#0,999542..."
4,2014-10-20,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,999912120917965|7#0,999379..."
...,...,...,...,...
1313,2020-01-15,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999954181295717|7#0,9996734746..."
1314,2020-01-16,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999952145103623|7#0,9996608774..."
1315,2020-01-17,1700,CurvaEfectiva_CLP,"Plazo#Valor|3#0,999873380808733|7#0,9996998671..."
1316,2020-01-20,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999952044955167|7#0,9996596504..."


In [27]:
vector_dias = [30, 90, 180, 360, 360*2, 360*3, 360*4, 360*5, 360*7, 360*9, 360*10, 360*15, 360*20, 360*30]


def evaluar_curva(pivotes, derivado):

    largo = len(pivotes)
    curvas = seleccionar_curva_derivados(derivado.flujos_valorizados["Moneda"][0])

    valores = []

    df = pd.DataFrame()
    df["Fechas"] = curvas["Fecha"]

    for i in range(largo):

        for j in range(len(curvas["Curva"])):

            valor_dia = pivotes[i]
            curva = curvas["Curva"][j]
            fecha_curva = curvas["Fecha"][j]
            curva_parseada = parsear_curva(curva, fecha_curva)
            valor = interpolacion_log_escalar(valor_dia, curva_parseada)
            valores.append(valor)
        
        df[str(vector_dias[i])] = valores
        valores = []

    
    return df


evaluar_curva(vector_dias, C)

,Fechas,30,90,180,360,720,1080,1440,1800,2520,3240,3600,5400,7200,10800
0,2014-10-07,0.997247,0.991653,0.982804,0.964742,0.936980,0.898953,0.857502,0.813820,0.728164,0.647457,0.612702,0.465844,0.350691,0.207265
1,2014-10-15,0.997160,0.991533,0.982841,0.965315,0.941714,0.907348,0.868270,0.826651,0.744382,0.660878,0.625129,0.482102,0.368499,0.223309
2,2014-10-16,0.996918,0.991607,0.983077,0.965602,0.942028,0.906980,0.867757,0.825204,0.743573,0.665230,0.626519,0.480286,0.366621,0.221611
3,2014-10-17,0.997158,0.991689,0.983067,0.965111,0.940740,0.905140,0.865408,0.822053,0.740999,0.659608,0.622917,0.476344,0.361477,0.216916
4,2014-10-20,0.997115,0.991639,0.983007,0.964917,0.940052,0.904730,0.865278,0.822271,0.740859,0.662270,0.623482,0.476765,0.361965,0.217350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,2020-01-15,0.998452,0.995842,0.991692,0.981892,0.964877,0.943027,0.916844,0.886522,0.823367,0.759320,0.729062,0.609334,0.509852,0.363745
1314,2020-01-16,0.998433,0.995735,0.991830,0.981989,0.965197,0.943235,0.917672,0.887917,0.825427,0.761611,0.731738,0.612373,0.516362,0.370894
1315,2020-01-17,0.998476,0.996076,0.992045,0.982662,0.965368,0.943788,0.918932,0.889901,0.829413,0.765411,0.735738,0.617512,0.522109,0.377115
1316,2020-01-20,0.998486,0.995838,0.991723,0.982295,0.965200,0.943424,0.918130,0.889492,0.829008,0.763974,0.734000,0.614916,0.519875,0.374718


In [28]:


def calcular_retornos(dfHistorico):

    numero_filas = len(vector_dias)
    numero_columnas = len(dfHistorico["30"])
    valores = []
    df = pd.DataFrame()


    for i in range(numero_filas):

        columna = dfHistorico[str(vector_dias[i])]

        for j in range(numero_columnas):

            if j == 0:

                valores.append(0)

            else:

                valor = np.log(columna[j] / columna[j-1])
                valores.append(valor)

        df[str(vector_dias[i])] = valores
        valores = []

    return df
        
a = evaluar_curva(vector_dias, C)
b = calcular_retornos(a)
b

,30,90,180,360,720,1080,1440,1800,2520,3240,3600,5400,7200,10800
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000088,-0.000121,0.000037,0.000594,0.005040,0.009295,0.012478,0.015643,0.022027,0.020516,0.020079,0.034305,0.049531,0.074557
2,-0.000242,0.000074,0.000240,0.000298,0.000333,-0.000406,-0.000590,-0.001752,-0.001086,0.006564,0.002220,-0.003775,-0.005110,-0.007632
3,0.000240,0.000083,-0.000010,-0.000509,-0.001368,-0.002031,-0.002711,-0.003826,-0.003469,-0.008488,-0.005765,-0.008241,-0.014129,-0.021413
4,-0.000043,-0.000050,-0.000061,-0.000201,-0.000732,-0.000452,-0.000151,0.000265,-0.000189,0.004027,0.000907,0.000883,0.001348,0.001997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,0.000027,0.000294,0.000176,0.000411,0.000425,0.001162,0.001877,0.002487,0.002742,0.002168,0.002310,0.006996,0.004421,0.006272
1314,-0.000019,-0.000107,0.000139,0.000099,0.000332,0.000221,0.000902,0.001572,0.002498,0.003012,0.003664,0.004975,0.012686,0.019463
1315,0.000043,0.000343,0.000217,0.000684,0.000177,0.000586,0.001373,0.002232,0.004818,0.004977,0.005451,0.008356,0.011068,0.016635
1316,0.000010,-0.000240,-0.000325,-0.000373,-0.000175,-0.000386,-0.000873,-0.000460,-0.000488,-0.001879,-0.002364,-0.004212,-0.004288,-0.006377


In [29]:


def volatilidades_derivados(dfRetornos):

    numero_filas = len(vector_dias)
    numero_columnas = len(dfRetornos[str(vector_dias[0])])
    df = pd.DataFrame()
    df["Pivotes"] = vector_dias
    valores = []

    for i in range(numero_filas):

        retornos = dfRetornos[str(vector_dias[i])]
        valor = ewma(retornos, 0.94)
        valores.append(valor["Vol c/ajuste"].values[0])
    
    df["Volatilidades"] = valores
    return df


d = volatilidades_derivados(b)
d



,Pivotes,Volatilidades
0,30,0.000183
1,90,0.000311
2,180,0.000516
3,360,0.000666
4,720,0.000814
5,1080,0.001300
6,1440,0.001902
7,1800,0.002539
8,2520,0.003781
9,3240,0.004794


In [30]:

def correlaciones_derivador(dfRetornos, dfVolatilidades):

    lenght = len(vector_dias)
    volatilidad = dfVolatilidades["Volatilidades"]
    corr = ewma_new_new_pivotes(lenght, dfRetornos, volatilidad)
    return corr

correlaciones_derivador(b, d)


,30,90,180,360,720,1080,1440,1800,2520,3240,3600,5400,7200,10800
30,0.000183,0.689280,0.657801,0.636162,0.394350,0.381897,0.375390,0.340184,0.310542,0.291301,0.287223,0.279931,0.282977,0.282184
90,0.689280,0.000311,0.815853,0.848922,0.485605,0.518657,0.510498,0.479363,0.467508,0.442045,0.426455,0.435801,0.445947,0.445453
180,0.657801,0.815853,0.000516,0.913545,0.609171,0.603132,0.607033,0.570512,0.530489,0.520502,0.515916,0.504264,0.517396,0.516914
360,0.636162,0.848922,0.913545,0.000666,0.587282,0.616024,0.616004,0.581994,0.560265,0.543116,0.530234,0.522193,0.535853,0.535382
720,0.394350,0.485605,0.609171,0.587282,0.000814,0.977712,0.952465,0.928546,0.892883,0.867288,0.857624,0.853106,0.834563,0.832127
1080,0.381897,0.518657,0.603132,0.616024,0.977712,0.001300,0.983674,0.963362,0.931791,0.914614,0.902319,0.904422,0.878545,0.875726
1440,0.375390,0.510498,0.607033,0.616004,0.952465,0.983674,0.001902,0.987455,0.956987,0.946943,0.938910,0.935381,0.908047,0.905144
1800,0.340184,0.479363,0.570512,0.581994,0.928546,0.963362,0.987455,0.002539,0.975503,0.961439,0.956004,0.949865,0.924560,0.921703
2520,0.310542,0.467508,0.530489,0.560265,0.892883,0.931791,0.956987,0.975503,0.003781,0.972586,0.964679,0.959185,0.934046,0.931178
3240,0.291301,0.442045,0.520502,0.543116,0.867288,0.914614,0.946943,0.961439,0.972586,0.004794,0.995937,0.984286,0.966825,0.964311


In [31]:
def covarianzas_derivador(dfRetornos, dfVolatilidades):

    lenght = len(vector_dias)
    volatilidad = dfVolatilidades["Volatilidades"]
    corr = covarianza_pivotes(lenght, dfRetornos, volatilidad)
    return corr


covarianzas_derivador(b, d)

,30,90,180,360,720,1080,1440,1800,2520,3240,3600,5400,7200,10800
30,3.350648e-08,3.919231e-08,6.210540e-08,7.758939e-08,5.874450e-08,9.090847e-08,1.306947e-07,1.580763e-07,2.149081e-07,2.556170e-07,2.839165e-07,4.234739e-07,5.985916e-07,8.990560e-07
90,3.919231e-08,9.648987e-08,1.307143e-07,1.757029e-07,1.227568e-07,2.095150e-07,3.016096e-07,3.780012e-07,5.490330e-07,6.582485e-07,7.153538e-07,1.118769e-06,1.600807e-06,2.408425e-06
180,6.210540e-08,1.307143e-07,2.660363e-07,3.139574e-07,2.556998e-07,4.045542e-07,5.955156e-07,7.470046e-07,1.034462e-06,1.286991e-06,1.436997e-06,2.149510e-06,3.083951e-06,4.640652e-06
360,7.758939e-08,1.757029e-07,3.139574e-07,4.439567e-07,3.184475e-07,5.337798e-07,7.806645e-07,9.844128e-07,1.411342e-06,1.734787e-06,1.907852e-06,2.875492e-06,4.126012e-06,6.209034e-06
720,5.874450e-08,1.227568e-07,2.556998e-07,3.184475e-07,6.622807e-07,1.034728e-06,1.474282e-06,1.918282e-06,2.747160e-06,3.383508e-06,3.768986e-06,5.737663e-06,7.848640e-06,1.178694e-05
1080,9.090847e-08,2.095150e-07,4.045542e-07,5.337798e-07,1.034728e-06,1.691175e-06,2.433076e-06,3.180327e-06,4.581218e-06,5.701838e-06,6.336666e-06,9.720230e-06,1.320299e-05,1.982225e-05
1440,1.306947e-07,3.016096e-07,5.955156e-07,7.806645e-07,1.474282e-06,2.433076e-06,3.617601e-06,4.767775e-06,6.881522e-06,8.634097e-06,9.643636e-06,1.470314e-05,1.995874e-05,2.996528e-05
1800,1.580763e-07,3.780012e-07,7.470046e-07,9.844128e-07,1.918282e-06,3.180327e-06,4.767775e-06,6.444309e-06,9.362354e-06,1.170019e-05,1.310553e-05,1.992788e-05,2.712300e-05,4.072580e-05
2520,2.149081e-07,5.490330e-07,1.034462e-06,1.411342e-06,2.747160e-06,4.581218e-06,6.881522e-06,9.362354e-06,1.429343e-05,1.762703e-05,1.969507e-05,2.996965e-05,4.080852e-05,6.127615e-05
3240,2.556170e-07,6.582485e-07,1.286991e-06,1.734787e-06,3.383508e-06,5.701838e-06,8.634097e-06,1.170019e-05,1.762703e-05,2.298083e-05,2.578229e-05,3.899558e-05,5.356058e-05,8.046195e-05


In [32]:
def crear_pivotes(fecha_inicial, pivotes):


    lenght = len(pivotes)
    dias = []
    for i in range(lenght):

        dia = add_days(fecha_inicial, pivotes[i])
        dias.append(dia)

    return dias

def buscar_pivotes(fecha_pivotes, fecha):

    lenght = len(fecha_pivotes)

    for i in range(lenght):

        fecha_probable = fecha_pivotes[i]
        if (i == 0 and fecha < fecha_probable) or (i == lenght - 1):

            return [i, i]

        elif fecha < fecha_probable:

            return [i-1, i]
        

def factor_desct_pivotes(pivotes, fecha_pivotes, moneda):


    dia_inical = add_days(fecha_pivotes[0], -pivotes[0])

    curva = ("SELECT * FROM [dbDerivados].[dbo].[TdCurvasDerivados] WHERE Tipo = 'CurvaEfectiva_"+moneda+"' AND Fecha = '"+str(dia_inical)+"'")
    curva = pd.read_sql(curva, cnn)
    curva = parsear_curva(curva["Curva"][0], dia_inical)

    lenght = len(pivotes)
    valor_factor = []
    df = pd.DataFrame()

    for i in range(lenght):
        
        valor = interpolacion_log_escalar(pivotes[i], curva)
        valor_factor.append(valor)

    df["Fecha"] = fecha_pivotes
    df["Pivote"] = pivotes
    df["FactorDescuento"] = valor_factor

    return df


fecha = datetime.date(2018, 4, 18)
pivotes_dias = crear_pivotes(fecha, vector_dias)
tabla = factor_desct_pivotes(vector_dias, pivotes_dias, "CLP")
tabla

,Fecha,Pivote,FactorDescuento
0,2018-05-18,30,0.998519
1,2018-07-17,90,0.994028
2,2018-10-15,180,0.986992
3,2019-04-13,360,0.971358
4,2020-04-07,720,0.943830
5,2021-04-02,1080,0.910173
6,2022-03-28,1440,0.875308
7,2023-03-23,1800,0.838749
8,2025-03-12,2520,0.768484
9,2027-03-02,3240,0.697169


In [33]:
def valor_alfa(fecha_inicial, fecha_pivote1, fecha_pivote2, fecha_pago):

    D_flujo = diferencia_dias_convencion("ACT360", fecha_inicial, fecha_pago)/360
    D_pivote1 = diferencia_dias_convencion("ACT360", fecha_inicial, fecha_pivote1)/360
    D_pivote2 = diferencia_dias_convencion("ACT360", fecha_inicial, fecha_pivote2)/360
    calculo = (D_flujo - D_pivote1)/(D_pivote2 - D_pivote1)
    return calculo


def interpolar_factorDesct(alfa_0, dfPivotes, fecha_pago):

    factores_descuento = dfPivotes["FactorDescuento"]
    numero_pivotes = buscar_pivotes(dfPivotes["Fecha"], fecha_pago)
    factor_pivote1 = factores_descuento[numero_pivotes[0]]
    factor_pivote2 = factores_descuento[numero_pivotes[1]]
    
    return alfa_0*factor_pivote1 + (1 - alfa_0)*factor_pivote2

def interpolar_volatilidad(alfa_0, dfPivotes, fecha_pago):

    volatilidades = dfPivotes["Volatilidades"]
    numero_pivotes = buscar_pivotes(dfPivotes["Fecha"], fecha_pago)
    volatilidad_pivote1 = volatilidades[numero_pivotes[0]]
    volatilidad_pivote2 = volatilidades[numero_pivotes[1]]

    return alfa_0*volatilidad_pivote1 + (1 - alfa_0)*volatilidad_pivote2
"""
tabla = generar_tabla_completa(vector_dias, fecha_fictisia, "CLP", C)
fecha_fictisia = datetime.date(2019, 8, 12)
indices = buscar_pivotes(pivotes_dias, fecha_fictisia)
fecha_inicial = add_days(pivotes_dias[0], -30)
fecha_pivote1 = pivotes_dias[indices[0]]
fecha_pivote2 = pivotes_dias[indices[1]]
alfa_0 = valor_alfa(fecha_inicial, fecha_pivote1, fecha_pivote2, fecha_fictisia)
interpolar_factorDesct(alfa_0, tabla, fecha_fictisia)
interpolar_volatilidad(alfa_0, tabla, fecha_fictisia)
"""
fecha_fictisia = datetime.date(2019, 8, 12)



In [34]:
def generar_tabla_completa(pivotes, fecha_inicial, moneda, derivado_ej):

    fecha_pivotes = crear_pivotes(fecha_inicial, pivotes)
    factores_desct = factor_desct_pivotes(pivotes, fecha_pivotes, moneda)
    historico = evaluar_curva(pivotes, derivado_ej)
    retorno = calcular_retornos(historico)
    volatilidades = volatilidades_derivados(retorno)
    factores_desct["Volatilidades"] = volatilidades["Volatilidades"]
    return factores_desct



generar_tabla_completa(vector_dias, fecha_fictisia, "CLP", C)




,Fecha,Pivote,FactorDescuento,Volatilidades
0,2019-09-11,30,0.998262,0.000183
1,2019-11-10,90,0.995377,0.000311
2,2020-02-08,180,0.991636,0.000516
3,2020-08-06,360,0.983755,0.000666
4,2021-08-01,720,0.965052,0.000814
5,2022-07-27,1080,0.945682,0.001300
6,2023-07-22,1440,0.923846,0.001902
7,2024-07-16,1800,0.899828,0.002539
8,2026-07-06,2520,0.849093,0.003781
9,2028-06-25,3240,0.796699,0.004794


In [35]:
def solucion_ecuacion(sigma_flujo, sigma_pivote1, sigma_pivote2, ro):

    """
    Funcion para calcular la solucion de la ecuacion propuesta
    para obtener el valor de alfa necesario para los calculos futuros
    :param sigma_flujo: Corresponde a la volatilidad del flujo
    :param sigma_pivote1: Corresponde a la volatilidad del pivote1
    :param sigma_pivote2: Correponde a la volatilidad del pivote2
    :param ro: Correspode a la correlacion del pivote1 y pivote2

    """

    A = (sigma_pivote1**2 + sigma_pivote2**2 - 2*ro*sigma_pivote1*sigma_pivote2)
    B = (2 * ro * sigma_pivote1* sigma_pivote2 - 2*sigma_pivote2**2)
    C = (sigma_pivote2**2 - sigma_flujo**2)

    x1 = (-B+math.sqrt(B**2-(4*A*C)))/(2*A)  # Fórmula de Bhaskara parte positiva
    x2 = (-B-math.sqrt(B**2-(4*A*C)))/(2*A)  # Fórmula de Bhaskara parte negativa

    return[x1, x2]

In [64]:
def calculo(derivado_IDKEY, fecha_valorizacion):

    derivado_ej = extrar_crear_derivado(derivado_IDKEY[0])
    derivado_ej.genera_flujos()
    derivado_ej.valoriza_flujos()
    tabla_total = generar_tabla_completa(vector_dias, fecha_valorizacion, derivado_ej.flujos_valorizados["Moneda"][0], derivado_ej)
    historico = evaluar_curva(vector_dias, derivado_ej)
    retornos = calcular_retornos(historico)
    correlacion = correlaciones_derivador(retornos, tabla_total)

    fechas_pivotes = tabla_total["Fecha"]

    pivotes = crear_pivotes(fecha_valorizacion, vector_dias)

    resultado = []

    largo_derivados = len(derivado_IDKEY)
    for j in range(largo_derivados):

        derivado = extrar_crear_derivado(derivado_IDKEY[j])
        derivado.genera_flujos()
        derivado.valoriza_flujos()
        tabla_derivado = derivado.flujos_valorizados[["ID","ActivoPasivo", "Fecha", "FechaFixing", "FechaFlujo", "FechaPago", "Flujo", "ValorPresenteMonFlujo", "Moneda", "MonedaBase"]]

        curva = seleccionar_curva_fecha(derivado.flujos_valorizados["Moneda"][0], str(fecha_valorizacion))
        curva_parseada = parsear_curva(curva["Curva"][0], fecha_valorizacion)

        fechas_pago = tabla_derivado["FechaPago"]
        lenght = len(fechas_pago)
        volatilidades_pivotes = tabla_total["Volatilidades"]

        proyeccion_pivotes = np.zeros(len(pivotes))


        for i in range(lenght):

            fecha_pago_actual = fechas_pago[i]
            indices_pivotes = buscar_pivotes(fechas_pivotes, fecha_pago_actual)
            alfa = valor_alfa(fecha_valorizacion, fechas_pivotes[indices_pivotes[0]], fechas_pivotes[indices_pivotes[1]], fecha_pago_actual)
            volatilidad = interpolar_volatilidad(alfa, tabla_total, fecha_pago_actual)
            factor_desct = interpolar_factorDesct(alfa, tabla_total, fecha_pago_actual)
            dia_pivote1 = vector_dias[indices_pivotes[0]]
            dia_pivote2 = vector_dias[indices_pivotes[1]]
            a =solucion_ecuacion(volatilidad, volatilidades_pivotes[indices_pivotes[0]], volatilidades_pivotes[indices_pivotes[1]],\
                                                                             correlacion[str(dia_pivote1)][str(dia_pivote2)] )
            factor = min(a)
            flujo1 = tabla_derivado["Flujo"][i]
            diferencia_dias = diferencia_dias_convencion("ACT360", fecha_valorizacion, fecha_pago_actual)
            factor_descuento = interpolacion_log_escalar(diferencia_dias, curva_parseada)
            VP = factor_descuento*flujo1
       
            proyeccion_pivotes[indices_pivotes[0]] += factor*factor_descuento*flujo1
            proyeccion_pivotes[indices_pivotes[1]] += (1 - factor)*factor_descuento*flujo1

        resultado.append(proyeccion_pivotes)
    
    return resultado





uwu = calculo(["146854", "146885", "146879", "146858", "146883"], fecha)

..\Derivados\DerivadosTipos\DerivadosSCC.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_cartera.MonedaActivo[0] = "CLP"


TypeError: '>' not supported between instances of 'datetime.date' and 'str'

In [63]:

uwu





[array([0.00000000e+00, 1.51417936e+07, 3.45689737e+07, 7.98683069e+08,
        1.15844476e+09, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00]),
 array([     0.        ,      0.        ,      0.        ,    811.62377692,
        143636.71056794,  48759.58324963,      0.        ,      0.        ,
             0.        ,      0.        ,      0.        ,      0.        ,
             0.        ,      0.        ]),
 array([     0.        ,      0.        ,      0.        ,   2434.87133077,
        430910.13170382, 146278.74974889,      0.        ,      0.        ,
             0.        ,      0.        ,      0.        ,      0.        ,
             0.        ,      0.        ]),
 array([0.00000000e+00, 0.00000000e+00, 1.33751313e+08, 2.06363662e+07,
        2.97931227e+09, 2.85189588e+09, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00